# Parcial Spark 10/5/21 - 1C 

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [12]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import random as rd

In [13]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Crear datasets

In [14]:
# dataframe 1
df = pd.DataFrame({
    'fecha':pd.date_range(start='2021-01-01',freq='H',periods=10000).strftime("%Y-%m-%d"),
    'dni': rd.choices(range(1000),k=10000),
    'id_localidad': rd.choices(range(25),k=10000),
    'resultado':rd.choices([0,1],k=10000)
})
tests = sqlContext.createDataFrame(df).rdd.map(tuple).cache()
tests.take(3)

[('2021-01-01', 971, 4, 1),
 ('2021-01-01', 660, 23, 0),
 ('2021-01-01', 503, 4, 0)]

In [15]:
# dataframe 2
df2 = pd.DataFrame({
    'id_localidad': range(25),
    'nombre':["localidad"+str(i) for i in range(25)] ,
    'provincia': rd.choices(['BA','CO','SF','OT'],k=25)
})
localidades = sqlContext.createDataFrame(df2).rdd.map(tuple).cache()
localidades.take(3)

[(0, 'localidad0', 'OT'), (1, 'localidad1', 'CO'), (2, 'localidad2', 'SF')]

Resolución del ejercicio
* 1)

In [44]:
tests_primer_trimestre = tests.filter(lambda x: pd.to_datetime('2021-01-01')<=pd.to_datetime(x[0])<pd.to_datetime('2021-04-01')).cache()
tests_primer_trimestre_join = tests_primer_trimestre.map(lambda x: (x[2],int(x[0].split("-")[1])))
# ->(id_localidad,mes)

In [45]:
def incrementaron_20_porc(x):
    """Recibe un diccionario del tipo {mes:cant_tests} donde mes es 1,2 o 3"""
    for i in range(2,4):
        if (i not in x) or (i-1 not in x):
            continue
        elif ((x[i] - x[i-1])/x[i-1]) > 0.2:
            return True
    return False

In [47]:
localidades.map(lambda x: (x[0],x[2])).join(tests_primer_trimestre_join)\
    .map(lambda x: (x[1],1))\
    .reduceByKey(lambda x,y: x+y)\
    .map(lambda x: (x[0][0],(x[0][1],x[1])))\
    .groupByKey().map(lambda x: (x[0],dict(x[1])))\
    .filter(lambda x: incrementaron_20_porc(x[1]))\
    .map(lambda x: x[0])

['SF']

Parte b)

In [54]:
rdd_positivos_localidad = localidades.map(lambda x: (x[0],x[1])).join(tests_primer_trimestre.map(lambda x: (x[2],x[3])))\
    .map(lambda x: (x[1][0],(x[1][1],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).cache()

[('localidad24', (40, 97)),
 ('localidad5', (42, 87)),
 ('localidad6', (56, 96)),
 ('localidad7', (44, 90)),
 ('localidad23', (57, 100)),
 ('localidad16', (36, 78)),
 ('localidad3', (44, 86)),
 ('localidad13', (38, 87)),
 ('localidad2', (56, 91)),
 ('localidad10', (47, 99))]

In [58]:
promedio_localidades = rdd_positivos_localidad.map(lambda x: (x[1][1],1)).reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))
promedio_localidades = promedio_localidades[0]/promedio_localidades[1]
promedio_localidades

86.4

In [67]:
rdd_positivos_localidad.filter(lambda x: x[1][1]>= promedio_localidades*0.3)\
    .reduce(lambda x,y: x if (x[1][0]/x[1][1])> y[1][0]/y[1][1] else y)[0]

'localidad2'